In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn_pandas import DataFrameMapper, cross_val_score
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import WordCount, CharCount, DataExtractor, ItemSelector, Vowels, Consonants, Polarity, Subjectivity, Nouns
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif

In [7]:
data = pd.read_csv('processed_posts.csv')
data.drop_duplicates('title', inplace=True)

data['gt20'] = data['ups'] > 20
# data['wordcount'] = data['title'].str.split().apply(len)
# data['charcount'] = data.title.str.len()
# data['vowels'] = data.title.str.findall(r'[aeiou]').apply(len)
# data['consonants'] = data.title.str.findall(r'[^aeiou]').apply(len)

train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gt20, 
                                                    test_size=0.20,
                                                    random_state=42)

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('wordcount', WordCount()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),
            
            ('polarity', Polarity()),
            
            ('subjectivity', Subjectivity()),
            
            ('noun_phrases', Nouns()),
            
            ])),
    ('clf', ExtraTreeClassifier()),
        ])

In [13]:
pipeline.fit(train_X, train_y)

ValueError: Input X must be non-negative

In [11]:
y = pipeline.predict(test_X)
accuracy_score(y_pred=y, y_true=test_y)

0.62553434588184309

In [ ]:
cross_val_score(pipeline, train_X, train_y, cv=5)

In [ ]:
# pipeline.predict(pd.Series(['I like Seinfield']))

In [ ]:
data.to_csv('gt_politics.csv', index=False)

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('wordcount', WordCount()),
            
            ('charcount', CharCount()),
            
            ('title', TfidfVectorizer(ngram_range=(1,4))),
            
            ('vowel', Vowels()),
            
            ('consonants', Consonants()),
            
            ('polarity', Polarity()),
            
            ('subjectivity', Subjectivity()),
            
            ('noun_phrases', Nouns()),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])